# Preparar notebook.

In [ ]:
#@title Instalar librerias y cargar conjunto de datos a limpiar { display-mode: "form" }
!pip install langdetect
import pandas as pd
import numpy as np
import pickle
from pickle import HIGHEST_PROTOCOL
from datetime import datetime
import matplotlib.pyplot as plot
from google.colab import drive
import plotly.express as px
import re
from langdetect import detect
from google.colab import data_table
from IPython.display import HTML, display
from IPython.display import clear_output 
from google.colab import output

drive.mount('/content/drive')
data_path = "/content/drive/" #@param {type:"string"}
FILE_NAME = "" #@param {type:"string"}

print('\nLeyendo archivo ',data_path + FILE_NAME + '.pkl')
df_original = pd.read_pickle(data_path + FILE_NAME + '.pkl')

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

clear_output()

# Información del conjunto de datos.

In [ ]:
#@title Estudiar columnas y tipos de datos. { display-mode: "form" }
df_original.info()

In [ ]:
#@title Estudiar valores más repetidos y su número de apariciones. { display-mode: "form" }
df_original['overview'].value_counts()[:20]

# Eliminar argumentos no útiles.

In [ ]:
#@title Remplazamos los elementos que solo contengan espacios vacios por NaN y borramos todos los NaN. { display-mode: "form" }
delete = {'overview','original_title'} #Columnas en las que se eliminan elementos con NaN
df_original.replace(r'^\s*$', np.nan, regex = True,inplace=True)
tam_ini = len(df_original)
df_original.dropna(subset=delete,inplace=True)
df_original.reset_index(drop=True,inplace=True)
tam_fin = len(df_original)
print("Se han borrado ", tam_ini - tam_fin, " elementos que tienen NaN en las columnas ",delete," del dataframe.\n")

In [ ]:
#@title Borramos los "No overview found.". { display-mode: "form" }
tam_ini = len(df_original)
df_original.drop(df_original[df_original.overview == "No overview found."].index, inplace=True)
df_original.reset_index(drop=True,inplace=True)
tam_fin = len(df_original)
print("Se han borrado ", tam_ini - tam_fin, " elementos que tienen 'No overview found.' en la columna ovewview del dataframe.\n")

In [ ]:
#@title Estudiar longitud de los overview. { display-mode: "form" }
df_original['overview'].str.split().\
  map(lambda x: len(x)).\
  hist(bins=100,figsize=(15,5),legend=True)

In [ ]:
#@title Borrar los elementos que tengan un argumento con un numero de palabras muy reducido. { display-mode: "form" }
too_short = 3
df_original['len_overview'] = [len(ov.split()) for ov in df_original.overview]
tam_ini = len(df_original)
short_mask = df_original['len_overview'] <= too_short
df_original.drop(df_original[short_mask].index, inplace=True)
df_original.reset_index(inplace=True, drop=True)
df_original.drop('len_overview', inplace=True, axis=1)
tam_fin = len(df_original)

print("Se han borrado ", tam_ini - tam_fin, " elementos del dataframe con longitud <= ",too_short)

In [ ]:
#@title Borrar sentencias que no están en inglés. (Aprox: 1h) { display-mode: "form" }
%%time
non_english_indices = list()
diferent_languages = list()
exc = list()
fin = len(df_original)

out = display(progress(0, len(df_original)), display_id=True)

for i, ov in enumerate(df_original.overview):
  try:
    out.update(progress(i, fin))
    lang=detect(ov)
    if lang != 'en' and lang !='uk':
      non_english_indices.append(i)
      if not(lang in diferent_languages):
        diferent_languages.append(lang)
  except Exception:
    exc.append(i)

df_non_english = pd.DataFrame(df_original.loc[non_english_indices],copy=True)
df_original.drop(non_english_indices, inplace=True)
df_original.reset_index(inplace=True, drop=True)

print("\nNumber of non english overviews: ", len(non_english_indices))
print("Diferent overview languages : ", len(diferent_languages))
print(diferent_languages,'\n\n')
print('Mostrando tabla con los elementos eliminados:\n')
data_table.DataTable(df_non_english, include_index=True, num_rows_per_page=5)


In [ ]:
#@title Guardar dataframe. { display-mode: "form" }

FILE_NAME = "similarity with bad overviews_31_08" #@param {type:"string"}

df_original.to_csv(data_path + FILE_NAME + '.csv',line_terminator='\r\n',index=False)

# Eliminar argumentos de peliculas similares a los no útiles ya conocidos - SMBC.

Usaremos el explorador general para encontrar las películas con argumentos que no son utiles y que no aportan información sobre las peliculas (como los que estan definidos en la variable 'bad_overviews' en la siguiente celda).

Así, una vez guardado el conjunto de datos procesado, se carga en el cuaderno de exploración para compararlo con con el conjunto de datos de 'bad_overviews' y generar la matriz de similitudes para saber que elementos eliminar del conjunto.

In [ ]:
#@title Generar dataframe de ovewrviews no útiles. { display-mode: "form" }
bad_overviews = ["No overview","No description","No synopsis","No plot","NaN","there is no information",
                "None","No available","No disponible","unavailable","Overview Coming Soon","No plot found anywhere online",
                "unknown","A plot is not available","please add plot","no summary available","Not available at this time",
                "What the movie has in store for you, wait and watch this space for more updates", "not description","Plot unknown","Know what this is about?"]

#Guardamos dataframe para usarlo en el cuaderno del explorador
df_bad_overviews = pd.DataFrame()
df_bad_overviews['Bad_overviews'] = bad_overviews
df_bad_overviews.to_csv(data_path + "bad_overviews_dataset.csv",index=False)

In [ ]:
#@title Tras usar la comparación de dataframes en el SMBC cargamos la matriz de similitud obtenida. { display-mode: "form" }

MATRIX_FILE_NAME = "similarity with bad overviews_31_08.pkl" #@param {type:"string"}
MATRIX_FILE_NAME =  data_path + MATRIX_FILE_NAME
try:
  df_similares_bad_overviews = pd.read_pickle(MATRIX_FILE_NAME)
except Exception as e:
  print("Hay algún problema con el fichero indicado.")
  raise e
else:
  print("Matriz de similitud cargada correctamente.")

data_table.DataTable(df_similares_bad_overviews, include_index=True, num_rows_per_page=5)

In [ ]:
#@title Borramos los overviews con alta similitud a los bad_overviews. { display-mode: "form" }
#Utilizamos la información de la matriz para borrar.

threshold = 0.5
remove_overviews = list()
i=0
for cosenos in df_similares_bad_overviews['cosenos_compare']:
  j=0
  while j < len(cosenos) and (cosenos[j]>=threshold):
    ind = df_similares_bad_overviews['index_compare'][i][j]
    if not(ind in remove_overviews):
      remove_overviews.append(ind)
    j+=1
  i+=1

print(len(remove_overviews))

data_table.DataTable(pd.DataFrame(df_original.loc[remove_overviews,'overview']), include_index=True, num_rows_per_page=20)

df_original.drop(remove_overviews, inplace=True)
df_original.reset_index(inplace=True, drop=True)

# Generar tablas necesarias para la DB.

In [ ]:
#@title Generar tabla de géneros y actores. { display-mode: "form" }

#Contamos el numero de generos distintos que hay
diferent_genres = list()
for i, genres in enumerate(df_original['genres']):
    for genre in genres:
      if not(genre in diferent_genres):
        diferent_genres.append(genre)
print("Numero de generos diferentes : ", len(diferent_genres))
print(diferent_genres)

#Sustituimos los string de los generos por su id
new_column = []
for list_genres in df_original['genres']:
  new_column.append([diferent_genres.index(genre) for genre in list_genres])

df_original['genres'] = new_column

#Generamos csv para la base de datos que asocia id a genero
df_genres = pd.DataFrame({'genre_id': range(0, len(diferent_genres)),'genre':diferent_genres})

df_genres.to_csv(data_path + "genres_ids.csv",index=False)

df_original['actors_string'] = [','.join(actors) for actors in df_original['actors']] #Concatenamos los actores en un string
df_original['actors'] = df_original['actors'].apply(lambda x: np.nan if len(x)==0 else x) #Remplazar listas vacias por nan

In [ ]:
#@title Generar tabla de películas-géneros. { display-mode: "form" }
#creamos las cabeceras
genresHeader = ['id_movie']
for id_genre in df_genres['genre_id']:
      aux = 'genre_' + str(id_genre)
      genresHeader.append(aux)

#Creamos la tabla
moviesGenres = []
for index, row in df_original.iterrows():
    booleanArr = [False] * len(df_genres['genre_id'])

    for genre in row['genres']:
      booleanArr[genre] = True

    resArr = [row['id']] + booleanArr
    moviesGenres.append(resArr)

df_moviesGenres = pd.DataFrame(moviesGenres, columns= genresHeader)
df_moviesGenres.to_csv(data_path + "movies_genres.csv",index=False)

In [ ]:
#@title Guardar dataframe. { display-mode: "form" }

FILE_NAME = "Final_dataset" #@param {type:"string"}

df_original.to_csv(data_path + FILE_NAME + '.csv',line_terminator='\r\n',index=False)

# Preparar dataframe en español

In [ ]:
#@title Borrar películas que no están en español.
delete = {'overview_es'} #Columnas en las que se eliminan elementos con NaN
df_original.replace(r'^\s*$', np.nan, regex = True,inplace=True)
tam_ini = len(df_original)
df_original.dropna(subset=delete,inplace=True)
df_original.reset_index(drop=True,inplace=True)
tam_fin = len(df_original)
print("Se han borrado ", tam_ini - tam_fin, " elementos que tienen NaN en las columnas ",delete," del dataframe.\n")

In [ ]:
#@title Guardar dataframe - Español.

FILE_NAME = "Final_dataset_spanish" #@param {type:"string"}


df_original.to_csv(data_path + FILE_NAME + '.csv',line_terminator='\r\n',index=False)